### Using the model from the experiment that provided the best MSE value
Look up the results in MLFlow for the best experiment and retrieve model name and MLFlow run_id, then fill them in the appropriate cells in this notebook

In [ ]:
%run '../lib/imports.ipynb'

In [ ]:
modelTypeName = "LSTM_AE_valueMinMaxScaled"

In [ ]:
dataColumnName = 'valueMinMaxScaled'

In [ ]:
%run '../lib/utils_anomaly_detection.ipynb'

In [ ]:
%run '../lib/model_commons.ipynb'

In [ ]:
from pathlib import Path
import os, sys

dataFileList = !ls ../../data/rq2-valid/*msg-w-spikes*.csv
dataLatencyFileList =  !ls ../../data/rq2-valid/*avg-latency*.csv

In [ ]:
dataFileList

In [ ]:
pd.set_option('display.max_rows', 5000)
pd.pandas.set_option('display.max_columns', None)

In [ ]:
mlflow_run_id = '<get_the_run_id_from_MLFlow>'

In [ ]:
model_name = '<get_the_model_name_from_MLFlow>'

In [ ]:
mlflow.set_tracking_uri(MLFLOW_URI)
mlflow.set_registry_uri(MLFLOW_URI)

In [ ]:
#model = mlflow.tensorflow.load_model(f'models:/{model_name}/1')

model_params = mlflow.get_run(mlflow_run_id).to_dictionary()['data']['params']

In [ ]:
model_params

In [ ]:
T = int(model_params['time_window'])

In [ ]:
####
# Set the target environment to cluster or local. Default is local
####
run_local = True

In [ ]:
if run_local == False:
    print("Running on Cluster")
    from codeflare_sdk import TokenAuthentication
    from codeflare_sdk import generate_cert
    
    auth = TokenAuthentication(
        token = "<your_access_token>", # execute ocp whoami -t on the authenticated cluster to obtain the token
        server = "<OCP cluster API URL>",
        skip_tls = True
    )
    auth.login()
    
    # Create required TLS cert and export the environment variables to enable TLS
    generate_cert.generate_tls_cert('raycluster-complete', 'raycluster')
    generate_cert.export_env('raycluster-complete', 'raycluster')
    
    ray_endpoint = 'ray://raycluster-complete-head-svc.raycluster.svc.cluster.local:10001' # ensure your ray cluster URL is correct
    ray.shutdown()
    ray.init(address=ray_endpoint, logging_level=logging.ERROR, log_to_driver=False)
else:
    ray.shutdown()
    ray.init()    

In [ ]:
"""
 This function uses one Tensor formatted input from the X_test dataset in order to predict ahead by a number
 of given steps. Then it readjusts by using another true value from X_test (as the nex index) before starting
 a new prediction cycle
"""
def generate_nsteps_forecast_ae(x_test, y_test, nn_model, enc_model, pred_ahead):
    max_len = x_test.shape[0]
    y_pred = []
    last_x = x_test[0]
    dim_shape = last_x.shape[1]
    index = 0
    ae_errors_at_p = []
    se_errors_at_p = []    
    while len(y_pred) < max_len:
        sequence = 0
        while sequence < pred_ahead:
            try:
                x_crt_input = last_x.reshape(1, -1, dim_shape)
                p_vector = nn_model.predict(x_crt_input, verbose=0)
                p = p_vector[0,0] # 1x1 array -> scalar
            except:
                print(f'Prediction error for x={x_crt_input} at sequence={sequence} for start index={index} when pred_ahead={pred_ahead}')
                print(f'Model config was:{nn_model.get_config()}')
                p = 0
                
            # update the predictions list
            y_pred.append(p)

            #prepare new_x cointaing prediction p for encoding
            new_x = last_x[:,0] # the first column contains the original input x from X_train
            new_x = np.roll(new_x, -1)
            new_x[-1] = p
            
            try:
                x_vector_ = np.concatenate([new_x.reshape(1, -1, D), 
                                 enc_model.predict(new_x.reshape(1, -1, D), verbose=0)], 
                                axis = -1)
                x_ = x_vector_[0][0] # x_ is 3D we need only the last dimension as a row
            except:
                x_ = 0

            # make the new input
            last_x = np.roll(last_x, -1)
            last_x[-1] = x_
            
            # increase index for the next run
            sequence += 1

        index += sequence
        if index < max_len:
            last_x = x_test[index]
            ae_errors_at_p.append(np.absolute(p - y_test[index-1]))
            se_errors_at_p.append(np.square(p - y_test[index-1]))
            #print(f"Arrived at index = {index} of {max_len} with value X={last_x}")
    
    if len(y_pred) > max_len:
        # predicted too much, cutoff the tail
        y_pred = y_pred[0:max_len]
        
    y_pred_array = np.array(y_pred)
    avg_ae = np.mean(ae_errors_at_p)
    avg_se = np.mean(se_errors_at_p)
    return (y_pred_array, avg_ae, avg_se)

In [ ]:
@ray.remote
def run_new_data_evaluation_ae(model_name, autoencoder_name, run_id, T, predict_ahead):
    import mlflow

    mlflow.set_tracking_uri(MLFLOW_URI)
    mlflow.set_registry_uri(MLFLOW_URI)

    model = mlflow.tensorflow.load_model(f'models:/{model_name}/1')
    autoencoder = mlflow.tensorflow.load_model(f'models:/{autoencoder_name}/1')

    params = mlflow.get_run(run_id).to_dictionary()['data']['params']
    n_neurons = params['n_layer_size']
    learning_rate = params['learning_rate']
    n_epochs = params['epochs']
    model_exp = params['model_exp']
    
    n_step_metrics = {}
    
    X_train, Y_train, X_test, Y_test = prepare_dataset(dataFrame[dataColumnName], T)
    
    #autoencode input
    X_test_ = np.concatenate([X_test, autoencoder.predict(X_test, verbose=0)], axis = -1)

    n_step_metrics = {}
    one_step_metrics = {}
    
    y_predict = model.predict(X_test_, verbose=0)
    errors_ae = calculate_absolute_prediction_errors(Y_test, y_predict)    
    errors_se = calculate_squared_prediction_errors(Y_test, y_predict)
    
    y_pred_nsteps, avg_ae, avg_se = generate_nsteps_forecast_ae(X_test_, Y_test, model, autoencoder, predict_ahead)    
    errors_ae2 = calculate_absolute_prediction_errors(Y_test, y_pred_nsteps)
    errors_se2 = calculate_squared_prediction_errors(Y_test, y_pred_nsteps)

    try:
        r2 = r2_score(Y_test, y_predict)
    except:
        r2 = 110
    if np.isnan(r2):
        r2 = 110

    try:
        mae = mean_absolute_error(Y_test, y_predict)
    except:
        mae = 100
    if np.isnan(mae):
        mae = 100

    try:
        mape = mean_absolute_percentage_error(Y_test, y_predict)
    except:
        mape = 100
    if np.isnan(mape):
        mape = 100

    try:
        mse = mean_squared_error(Y_test, y_predict)
    except:
        mse = 100
    if np.isnan(mse):
        mse = 100
    
    try:
        pcc = np.corrcoef(Y_test, y_predict.flatten())[0,1]
    except:
        pcc = 100
    if np.isnan(pcc):
        pcc = 100

    one_step_metrics = {
                        'r2_1Step': r2,
                        'mae_1Step': mae,
                        'mape_1Step': mape,
                        'mse_1Step': mse,
                        'pcc_1Step': pcc
    }
    
    try:
        r2_nStep = r2_score(Y_test, y_pred_nsteps)
    except:
        r2_nStep = 100

    try:
        mae_nStep = mean_absolute_error(Y_test, y_pred_nsteps)
    except:
        mae_nStep = 100

    try:
        mape_nStep = mean_absolute_percentage_error(Y_test, y_pred_nsteps)
    except:
        mape_nStep = 100

    try:
        mse_nStep = mean_squared_error(Y_test, y_pred_nsteps)
    except:
        mse_nStep = 100

    try:
        pcc_nStep = np.corrcoef(Y_test, y_pred_nsteps.flatten())[0,1]
    except:
        pcc_nStep = 100

    crt_step = f'predict_ahead_{predict_ahead}'
    n_step_metrics[crt_step] = {
                                    'r2_nStep': r2_nStep,
                                    'mae_nStep': mae_nStep,
                                    'mape_nStep': mape_nStep,
                                    'mse_nStep': mse_nStep,
                                    'pcc_nStep': pcc_nStep,
                                    'agv_ae_at_nStep': avg_ae,
                                    'agv_se_at_nStep': avg_se
                                   }

    with mlflow.start_run(run_id=run_id, nested=True) as run:
        artifact_uri = run.info.artifact_uri
        three_sigma_thresholds = mlflow.artifacts.load_dict(artifact_uri + "/three_sigma_thresholds.json")        

        anomalies_ae = calculate_3sigma_anomalies(errors_ae, 
                                                   three_sigma_thresholds['lo_3sigma_ae'], 
                                                   three_sigma_thresholds['up_3sigma_ae'])
        anomalies_se = calculate_3sigma_anomalies(errors_se, 
                                                   three_sigma_thresholds['lo_3sigma_se'], 
                                                   three_sigma_thresholds['up_3sigma_se'])

        anomalies_ae2 = calculate_3sigma_anomalies(errors_ae2, 
                                                   three_sigma_thresholds['lo_3sigma_ae'], 
                                                   three_sigma_thresholds['up_3sigma_ae'])
        anomalies_se2 = calculate_3sigma_anomalies(errors_se2, 
                                                   three_sigma_thresholds['lo_3sigma_se'], 
                                                   three_sigma_thresholds['up_3sigma_se'])
        
        fig = plt.figure(figsize=(20,15))
        title = "Compare Forecasts for T=" + str(T) + " with predict 1 on "+ str(model_exp) + " for trial " + str(trial_fname)
        plt.title(title)
        plt.plot(y_predict,label="Predict 1-step Forecast", alpha=0.6, c='red', linewidth=3)
        plt.plot(Y_test,label="Original Data", alpha=0.6, c='black')
        plt.scatter(np.where(anomalies_ae==True), y_predict[np.where(anomalies_ae==True)], 
                    alpha=0.8, color='green', s=350, label="Anomalies AE")
        plt.scatter(np.where(anomalies_se==True), y_predict[np.where(anomalies_se==True)], 
                    alpha=0.8, color='magenta', s=150, label = "Anomalies SE")
        plt.legend();    
        figName = f"Y_predict_ahead-1-step-T_{T}-fname-{trial_fname}.png"
        mlflow.log_figure(fig, figName)
        fig.clf()
        plt.close()
        
        fig = plt.figure(figsize=(20,15))
        title = "Compare Forecasts for T=" + str(T) + " with predict_ahead= "+ str(predict_ahead) + " on model " + str(model_exp) + " for trial " + str(trial_fname)
        plt.title(title)
        plt.plot(y_pred_nsteps,label="Predict n-step Forecast", alpha=0.6, c='red', linewidth=3)
        plt.plot(Y_test,label="Original Data", alpha=0.6, c='black')
        plt.scatter(np.where(anomalies_ae2==True), y_pred_nsteps[np.where(anomalies_ae2==True)], 
                    alpha=0.8, color='green', s=350, label="Anomalies AE")
        plt.scatter(np.where(anomalies_se2==True), y_pred_nsteps[np.where(anomalies_se2==True)], 
                    alpha=0.8, color='magenta', s=150, label = "Anomalies SE")
        plt.legend();    
        figName = f"Y_predict_ahead-{predict_ahead}-step-T_{T}-fname-{trial_fname}.png"
        mlflow.log_figure(fig, figName)
        fig.clf()
        plt.close()        

        fname_nstep = f'{predict_ahead}-{trial_fname}-nstep-metric.json'
        fname_1step = f'{predict_ahead}-{trial_fname}-1step-metric.json'
        mlflow.log_dict(n_step_metrics, fname_nstep)
        mlflow.log_dict(one_step_metrics, fname_1step)
        
        result = {
            'one_step_metrics' : one_step_metrics,
            'n_step_metrics' : n_step_metrics
        }
        
    return result

In [ ]:
results_trials = {}
for filePos in range(len(dataFileList)):
    
    data = pd.read_csv(dataFileList[filePos], index_col='EventDateTime', parse_dates=['EventDateTime'])
    dataLatency = pd.read_csv(dataLatencyFileList[filePos], index_col='EventDateTime', parse_dates=['EventDateTime'])
    print(f'Processing files at position {filePos} in list')
    %run '../lib/prepareDataSet.ipynb'

    trial_fname = os.path.basename(dataFileList[filePos])
    results_predict_ahead = {}
    for predict_ahead in [5, 10, 15, 30, 45, 60, 90, 120]:
        res = run_new_data_evaluation_ae.remote(model_name, mlflow_run_id, T, predict_ahead, trial_fname)
        tag = f'predict_ahead_{predict_ahead}-trial_{trial_fname}'
        results_predict_ahead[tag] = res
        
    results_trials[trial_fname] = results_predict_ahead    

In [ ]:
results_trials

In [ ]:
data_trials = {}
for trial in results_trials.keys():
    data_trial = {}
    for item in results_trials[trial].keys():
        try:
            res = ray.get(results_trials[trial][item])
        except:
            print(f'Error getting results for key:{item}')
            res = None
        data_trial[item] = res
        
    data_trials[trial] = data_trial

In [ ]:
data_trials

In [ ]:
ray.shutdown()

In [ ]:
def find_best_result():
    min_mse = 1000
    tag = ''
    result = {}
    for trial in data_trials.keys():
        crt_trial = data_trials[trial]
        for predict_ahead in crt_trial.keys():
            crt_one_step = crt_trial[predict_ahead]['one_step_metrics']
            if crt_one_step['mse_1Step'] < min_mse:
                min_mse = crt_one_step['mse_1Step']
                tag = str(predict_ahead) + '-' + str(trial)
                result = crt_trial[predict_ahead]

    return result, tag
                

In [ ]:
find_best_result()